In [1]:
using Revise
using Scattering
using LinearAlgebra
using Test

┌ Info: Precompiling Scattering [a5c306d0-0a40-4008-a012-bc604123031d]
└ @ Base loading.jl:1260


# Rotation Representations

### References
1. http://www.yxliu.group/2020/04/scattering-4
2. https://en.wikipedia.org/wiki/Euler_angles
3. https://en.wikipedia.org/wiki/Rotation_matrix
4. https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation
5. https://en.wikipedia.org/wiki/Euler%27s_rotation_theorem
6. http://mathworld.wolfram.com/EulerAngles.html

## Some experiments on rotation operations

In [2]:
# u, v, w is the basis vector of UVW coordinate system with its elements expressed in the XYZ system
# i.e., in UVW coordinate system, u = [1, 0, 0], v = [0, 1, 0], w = [0, 0, 1]
# Let w points to the principle direction of the particle.
w = [1.0, 2.0, 3.0]
normalize!(w)
a = [0, 1, 2]
u = cross(w, a)
normalize!(u)
v = cross(w, u)
normalize!(v)
w

3-element Array{Float64,1}:
 0.2672612419124244
 0.5345224838248488
 0.8017837257372732

In [3]:
# P is the rotation matrix which rotates XYZ coordinate system (basis vectors) to UVW coordinate system
# [u v w] = [e_x e_y e_z]P
P = hcat(u, v, w)

3×3 Array{Float64,2}:
  0.408248   0.872872  0.267261
 -0.816497   0.218218  0.534522
  0.408248  -0.436436  0.801784

In [4]:
# R is the rotation matrix which rotates a vector in XYZ system to UVW coordinate system
# R = P^{-1} = P^T
# R * u = [1, 0, 0], R * v = [0, 1, 0], R * w = [0, 0, 1]
@test transpose(P) ≈ inv(P)
R = transpose(P)

3×3 Transpose{Float64,Array{Float64,2}}:
 0.408248  -0.816497   0.408248
 0.872872   0.218218  -0.436436
 0.267261   0.534522   0.801784

In [5]:
@test det(R) ≈ 1

Test Passed

In [6]:
# Verify that R indeed transforms basis vector of in the XYZ coordinate to u, v, w.
# u, v, w have components expressed in the XYZ coordinate.
@test R * u ≈ [1, 0, 0]
@test R * v ≈ [0, 1, 0]
@test R * w ≈ [0, 0, 1] 

Test Passed

In [7]:
tr(R)

1.4282499064371286

In [8]:
# compute the rotation angle
θ = acos((tr(R)-1)/2)
rad2deg(θ)

77.63580469304388

In [9]:
# compute the rotation axis expressed in the XYZ system.
uu = [R[3,2]-R[2,3], R[1,3]-R[3,1], R[2,1]-R[1,2]] / (2sin(θ))

3-element Array{Float64,1}:
 0.49700656431759865
 0.07216735383016143
 0.8647406247345901

In [10]:
# another way to compute the rotation axis
# Note that the result may be different from the one computed above with only a sign.
vals, vecs = eigen(R)
idx = findfirst(x -> x ≈ one(x), vals)
uu2 = real(vecs[:, idx])
@test uu2 ≈ uu || uu2 ≈ -uu

Test Passed

In [11]:
# another way to compute the rotation angle
vv = [R[3,2]-R[2,3], R[1,3]-R[3,1], R[2,1]-R[1,2]]
@test asin(norm(vv)/2) ≈ θ

Test Passed

In [12]:
# a proper rotation matrix should have det(R) = 1
det(R)

1.0000000000000002

In [13]:
transpose(R)

3×3 Array{Float64,2}:
  0.408248   0.872872  0.267261
 -0.816497   0.218218  0.534522
  0.408248  -0.436436  0.801784

In [14]:
inv(R)

3×3 Transpose{Float64,Array{Float64,2}}:
  0.408248   0.872872  0.267261
 -0.816497   0.218218  0.534522
  0.408248  -0.436436  0.801784

In [15]:
# a proper rotation matrix should be an orthogonal matrix
@test transpose(R) ≈ inv(R)

Test Passed

In [16]:
# convert current u, v, w representation to an Euler angle representation.
# Convention used: Z1Y2Z3

# Procedure:
# 1. Rotate about +z by eta (counter-clockwise in x-y plane)
# 2. Rotate about the former y-axis (which is y'), counter clockwise in x'-z plane. then
# 3. Rotate about the former z-axis (which is z'), counter-clockwise in x'-y' plane

# See wiki page:
# 1. https://en.wikipedia.org/wiki/Rotation_matrix
# 2. https://en.wikipedia.org/wiki/Euler_angles#Intrinsic_rotations
α = atan(R[2,3], R[1,3])
β = acos(R[3,3])
γ = atan(R[3,2], -R[3,1])
rad2deg.([α, β, γ])
c1 = cos(α)
c2 = cos(β)
c3 = cos(γ)
s1 = sin(α)
s2 = sin(β)
s3 = sin(γ)

Ra = zero(R)
Ra[1, 1] = c1*c2*c3 - s1*s3
Ra[1, 2] = -c3*s1 - c1*c2*s3
Ra[1, 3] = c1 * s2
Ra[2, 1] = c1*s3 + c2*c3*s1
Ra[2, 2] = c1*c3 - c2*s1*s3
Ra[2, 3] = s1*s2
Ra[3, 1] = -c3*s2
Ra[3, 2] = s2*s3
Ra[3, 3] = c2
@test Ra ≈ R

Test Passed

In [17]:
rad2deg.([α, β, γ])

3-element Array{Float64,1}:
 -46.91127686463718
  36.69922520048988
 116.56505117707799

In [18]:
# Convention: Z1X2Z3
α = atan(R[1,3], -R[2,3])
β = acos(R[3,3])
γ = atan(R[3,1], R[3,2])
rad2deg.([α, β, γ])
c1 = cos(α)
c2 = cos(β)
c3 = cos(γ)
s1 = sin(α)
s2 = sin(β)
s3 = sin(γ)

Rb = zero(R)
Rb[1, 1] = c1*c3 - c2*s1*s3
Rb[1, 2] = -c1*s3 - c2*c3*s1
Rb[1, 3] = s1 * s2
Rb[2, 1] = c3*s1 + c1*c2*s3
Rb[2, 2] = c1*c2*c3 - s1*s3
Rb[2, 3] = -c1*s2
Rb[3, 1] = s2*s3
Rb[3, 2] = c3*s2
Rb[3, 3] = c2
@test Rb ≈ R

Test Passed

In [19]:
# Convention Z1Y2X3
α = atan(R[2,1],R[1,1])
β = asin(-R[3,1])
γ = atan(R[3,2], R[3,3])
rad2deg.([α, β, γ])
c1 = cos(α)
c2 = cos(β)
c3 = cos(γ)
s1 = sin(α)
s2 = sin(β)
s3 = sin(γ)

Z1Y2X3 = zero(R)
Z1Y2X3[1, 1] = c1*c2
Z1Y2X3[1, 2] = c1*s2*s3 - c3*s1
Z1Y2X3[1, 3] = s1*s3 + c1*c3*s2 
Z1Y2X3[2, 1] = c2*s1
Z1Y2X3[2, 2] = c1*c3 + s1*s2*s3
Z1Y2X3[2, 3] = c3*s1*s2 - c1*s3
Z1Y2X3[3, 1] = -s2
Z1Y2X3[3, 2] = c2*s3
Z1Y2X3[3, 3] = c2*c3
@test Z1Y2X3 ≈ R

Test Passed

In [20]:
# Convention: Z1Y2Z3
Rp = hcat([-1, 0, 0], [0, 0, 1], [0, 1, 0])
α = atan(Rp[2,3], Rp[1,3])
β = acos(Rp[3,3])
γ = atan(Rp[3,2], -Rp[3,1])
rad2deg.([α, β, γ])

3-element Array{Float64,1}:
 90.0
 90.0
 90.0

In [21]:
θp = acos((tr(Rp)-1)/2)
up =[Rp[3,2]-Rp[2,3], Rp[1,3]-Rp[3,1], Rp[2,1]-Rp[1,2]]
θpp = asin(norm(up)/2)
# normalize!(up)

rad2deg(θp), rad2deg(θpp), up

(180.0, 0.0, [0, 0, 0])

In [22]:
vals, vecs = eigen(Rp)
idx = findfirst(x -> x ≈ one(x), vals)
uu = real(vecs[:, idx])

3-element Array{Float64,1}:
 0.0
 0.7071067811865475
 0.7071067811865477

## Testing Scattering/rotation.jl

### Testing EulerAngle

In [34]:
using StaticArrays: SVector, FieldVector
α, β, γ = π/4, π/3, π/2
# constructors of EulerAngle
# initialize by a vector
EulerAngle([α, β, γ])

EulerAngle{Float64}(0.7853981633974483, 1.0471975511965976, 1.5707963267948966)

In [35]:
EulerAngle(1.3, 0, 0)

EulerAngle{Float64}(1.3, 0.0, 0.0)

In [36]:
# initialize by three angles
euler = EulerAngle(α, β, γ)

EulerAngle{Float64}(0.7853981633974483, 1.0471975511965976, 1.5707963267948966)

In [37]:
# copy constructor
EulerAngle(euler)

EulerAngle{Float64}(0.7853981633974483, 1.0471975511965976, 1.5707963267948966)

In [38]:
# initialize by a static vector
sv = SVector(1.0, 2.0, 3.0)
EulerAngle(sv)

EulerAngle{Float64}(1.0, 2.0, 3.0)

In [39]:
# typeof(euler) <: AbstractRotation{T} where {T<:Real}

### Testing RotationMatrix Constructors

In [40]:
# Follow the Z1Y2Z3 convention
α = atan(v[3], u[3])
β = acos(w[3])
γ = atan(w[2], -w[1])
rad2deg.([α, β, γ])

3-element Array{Float64,1}:
 -46.91127686463718
  36.69922520048988
 116.56505117707799

In [41]:
# constructor of RotationMatrix initialized with a rotation matrix
rotmat = RotationMatrix(Vector3D(u), Vector3D(v), Vector3D(w))
@test transpose(RotMatrix(hcat(u,v,w))) ≈ rotmat.R

Test Passed

In [42]:
# Convert an EulerAngle instance to a RotationMatrix instance
rotmat_euler = RotationMatrix(EulerAngle(α, β, γ))
@test rotmat_euler.R ≈ rotmat.R

Test Passed

In [43]:
# Convert a RotationMatrix instance to an EulerAngle instance
euler = EulerAngle(rotmat_euler)
@test [euler.α, euler.β, euler.γ] ≈ [α, β, γ]

Test Passed

### Testing EulerAxisAngle Constructors

In [44]:
# constructor of AxisAngleRepresentation initialized with a RotationMatrix instance
# conversion from rotation matrix representation to axis-angle representation
axisangle = EulerAxisAngle(rotmat)

EulerAxisAngle{Float64}([0.49700656431759865, 0.07216735383016143, 0.8647406247345901], 1.3550004093288814, [0.0 -0.8647406247345901 0.07216735383016143; 0.8647406247345901 0.0 -0.49700656431759865; -0.07216735383016143 0.49700656431759865 0.0])

In [45]:
# Convert Euler axis-angle representation to rotation matrix representation
rotmat_axisangle = RotationMatrix(axisangle)
@test rotmat_axisangle.R ≈ rotmat.R atol=1e-15

Test Passed

In [46]:
axisangle2 = EulerAxisAngle(axisangle.ω, axisangle.θ)
@test axisangle2.K ≈ axisangle.K

Test Passed

### Testing conversion and promotion

In [47]:
euler

EulerAngle{Float64}(-0.8187562376025611, 0.6405223126794245, 2.0344439357957027)

In [48]:
axisangle

EulerAxisAngle{Float64}([0.49700656431759865, 0.07216735383016143, 0.8647406247345901], 1.3550004093288814, [0.0 -0.8647406247345901 0.07216735383016143; 0.8647406247345901 0.0 -0.49700656431759865; -0.07216735383016143 0.49700656431759865 0.0])

In [49]:
EulerAxisAngle(euler)

EulerAxisAngle{Float64}([0.4970065643175987, 0.07216735383016142, 0.86474062473459], 1.3550004093288814, [0.0 -0.86474062473459 0.07216735383016142; 0.86474062473459 0.0 -0.4970065643175987; -0.07216735383016142 0.4970065643175987 0.0])

In [50]:
RotationMatrix(euler) |> EulerAxisAngle

EulerAxisAngle{Float64}([0.4970065643175987, 0.07216735383016142, 0.86474062473459], 1.3550004093288814, [0.0 -0.86474062473459 0.07216735383016142; 0.86474062473459 0.0 -0.4970065643175987; -0.07216735383016142 0.4970065643175987 0.0])

In [51]:
EulerAngle(axisangle)

EulerAngle{Float64}(-0.8187562376025609, 0.6405223126794247, 2.0344439357957027)

In [52]:
convert(EulerAngle, axisangle)

EulerAngle{Float64}(-0.8187562376025609, 0.6405223126794247, 2.0344439357957027)

In [53]:
convert(EulerAngle, euler)

EulerAngle{Float64}(-0.8187562376025611, 0.6405223126794245, 2.0344439357957027)

In [97]:
@test euler == axisangle

Test Passed

In [96]:
@test euler == euler

Test Passed

In [55]:
@test promote(euler) == rotmat

Test Passed

In [56]:
@test promote(axisangle) == rotmat

Test Passed

In [57]:
one(rotmat)

RotationMatrix{Float64}([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0])

In [58]:
one(euler)

EulerAngle{Float64}(0.0, 0.0, 0.0)

In [59]:
promote(one(euler))

RotationMatrix{Float64}([1.0 -0.0 0.0; 0.0 1.0 0.0; -0.0 0.0 1.0])

In [60]:
one(axisangle)

EulerAxisAngle{Float64}([1.0, 0.0, 0.0], 0.0, [0.0 -0.0 0.0; 0.0 0.0 -1.0; -0.0 1.0 0.0])

In [61]:
promote(one(axisangle))

RotationMatrix{Float64}([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0])

### Testing inv function

In [62]:
rot1 = rotmat

RotationMatrix{Float64}([0.408248290463863 -0.816496580927726 0.408248290463863; 0.8728715609439696 0.21821789023599242 -0.4364357804719848; 0.2672612419124244 0.5345224838248488 0.8017837257372732])

In [63]:
# Test inv function for RotationMatrix
irot1 = inv(rot1)
@test transpose(rot1.R) ≈ irot1.R

Test Passed

In [64]:
rot2 = EulerAxisAngle(rot1)

EulerAxisAngle{Float64}([0.49700656431759865, 0.07216735383016143, 0.8647406247345901], 1.3550004093288814, [0.0 -0.8647406247345901 0.07216735383016143; 0.8647406247345901 0.0 -0.49700656431759865; -0.07216735383016143 0.49700656431759865 0.0])

In [65]:
# Test inv function for EulerAxisAngle
irot2 = inv(rot2)
@test irot2 == irot1

Test Passed

In [66]:
rot3 = EulerAngle(rot1)

EulerAngle{Float64}(-0.818756237602561, 0.6405223126794245, 2.0344439357957027)

In [67]:
# Test inv function for EulerAngle
irot3 = inv(rot3)
@test irot3 == irot1

Test Passed

### Testing multiplication of two AbstractRotation instances.

In [68]:
# Test multiplication of two RotationMatrix instances.
R1 = rotmat
M = RotationMatrix(Rp)
R2 = R1 * M
@test R2.R ≈ R1.R*M.R

Test Passed

In [69]:
# Test multiplication of a EulerAxisAngle instance and a RotationMatrix instance.
R1 = rotmat
R1a = EulerAxisAngle(R1)
M = RotationMatrix(Rp)
R2 = R1a * M
@test R2 == R1 * M

Test Passed

In [70]:
# Test multiplication of a RotationMatrix instance and a EulerAxisAngle instance.
R1 = rotmat
M = RotationMatrix(Rp)
Ma = EulerAxisAngle(M)
R2 = R1 * Ma
@test R2 == R1 * M

Test Passed

In [71]:
# Test multiplication of a EulerAxisAngle instance and a EulerAxisAngle instance.
R1 = rotmat
R1a = EulerAxisAngle(R1)
M = RotationMatrix(Rp)
Ma = EulerAxisAngle(M)
R2 = R1a * Ma
@test R2 == R1 * M

Test Passed

### Testing computing power of a AbstractRotation instance.

In [72]:
function pow1(rot::AbstractRotation, n::Integer)
    rot = promote(rot)
    result = rot
    for i in 1:(n-1)
        result *= rot
    end
    result
end

pow1 (generic function with 1 method)

In [73]:
function pow(rot::RotationMatrix, n::Integer)
    if n == zero(n)
        return one(rot)
    elseif n == one(n)
        return rot
    else
        if isodd(n)
            return rot * pow(rot, n-1)
        else
            rothalf = pow(rot, n÷2)
            return rothalf * rothalf 
        end
    end
end

pow(rot::AbstractRotation, n::Integer) = pow(promote(rot), n)

pow (generic function with 2 methods)

In [74]:
pow(euler, 0)

RotationMatrix{Float64}([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0])

In [75]:
pow(euler, 1)

RotationMatrix{Float64}([0.40824829046386313 -0.8164965809277259 0.4082482904638629; 0.8728715609439694 0.21821789023599242 -0.4364357804719848; 0.26726124191242434 0.5345224838248487 0.8017837257372732])

In [76]:
pow(euler, 2)

RotationMatrix{Float64}([-0.4369210333151354 -0.2932896043722907 0.8503418245705543; 0.43018214432212887 -0.8983623348886722 -0.08881687880007882; 0.7899641342195538 0.32699590703939707 0.5186813505672173])

In [77]:
pow(euler, 3)

RotationMatrix{Float64}([-0.20711300761088602 0.7472703153125533 0.6314200487243415; -0.6322711178708509 -0.5947556020638506 0.4964866637886771; 0.7465503570320742 -0.29639981387703873 0.5956590591512387])

In [78]:
pow(euler, 4)

RotationMatrix{Float64}([0.7364715816744584 0.6696830270043788 0.09557328459445946; -0.644577211376976 0.651844177986726 0.3995239494677259; 0.2052555187063243 -0.35584239624735736 0.9117271308201462])

In [79]:
pow(euler, 5).R

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 0.910754   -0.404104   0.0850187
 0.412606    0.882094  -0.227304
 0.0168598   0.242097   0.970106

In [80]:
pow(euler, 100).R

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 -0.443094   0.414668   0.794807
 -0.277188  -0.906518   0.318422
  0.852546  -0.0792197  0.516614

In [81]:
euler^1

RotationMatrix{Float64}([0.40824829046386313 -0.8164965809277259 0.4082482904638629; 0.8728715609439694 0.21821789023599242 -0.4364357804719848; 0.26726124191242434 0.5345224838248487 0.8017837257372732])

In [82]:
euler^2

RotationMatrix{Float64}([-0.4369210333151354 -0.2932896043722907 0.8503418245705543; 0.43018214432212887 -0.8983623348886722 -0.08881687880007882; 0.7899641342195538 0.32699590703939707 0.5186813505672173])

In [83]:
euler^3

RotationMatrix{Float64}([-0.20711300761088602 0.7472703153125533 0.6314200487243415; -0.6322711178708509 -0.5947556020638506 0.4964866637886771; 0.7465503570320742 -0.29639981387703873 0.5956590591512387])

In [84]:
euler^4

RotationMatrix{Float64}([0.7364715816744584 0.6696830270043788 0.09557328459445946; -0.644577211376976 0.651844177986726 0.3995239494677259; 0.2052555187063243 -0.35584239624735736 0.9117271308201462])

In [85]:
@test pow(euler, 100).R ≈ (euler^100).R

Test Passed

In [86]:
using BenchmarkTools

In [87]:
@btime pow1($euler, 1000)

  142.561 μs (3004 allocations: 172.31 KiB)


RotationMatrix{Float64}([-0.17617351956438138 0.7712758102440354 0.6116342988556001; -0.6592241548072412 -0.553880454852226 0.50856656934094; 0.7310173764848875 -0.31360814126062625 0.606022713280731])

In [88]:
# the implementation of pow (or Scattering.^ which is the same as pow here)
# is significantly fast than the naive impolementation.
@btime $euler^1000

  2.327 μs (49 allocations: 3.02 KiB)


RotationMatrix{Float64}([-0.1761735195643819 0.7712758102440348 0.6116342988555798; -0.6592241548072477 -0.5538804548522176 0.5085665693409378; 0.7310173764848691 -0.3136081412606315 0.6060227132807026])

In [89]:
# @code_warntype euler^100

### Testing applying AbstractRotation instance on an vector.

In [90]:
# Test multiplication of a RotationMatrix instance and a Vector
M = RotationMatrix(Rp)
v = [1.0, 2.0, 3.0]
@test M*v ≈ M.R * v

Test Passed

In [91]:
# Test multiplication of a RotationMatrix instance and a RVector (SVector, QVector)
M = RotationMatrix(Rp)
r = RVector([1.0, 2.0, 3.0])
@test M*r ≈ M.R * r

Test Passed

In [92]:
# Test multiplication of a EulerAxisAngle instance and a Vector
M = RotationMatrix(Rp)
Ma = EulerAxisAngle(M)
v = [1.0, 2.0, 3.0]
@test Ma*v ≈ M.R * v

Test Passed

In [93]:
# Test multiplication of a EulerAxisAngle instance and a RVector (SVector, QVector)
M = RotationMatrix(Rp)
Ma = EulerAxisAngle(M)
v = RVector([1.0, 2.0, 3.0])
@test Ma*v ≈ M.R * v

Test Passed

In [94]:
# Test multiplication of a EulerAngle instance and a Vector
M = RotationMatrix(Rp)
Ma = EulerAngle(M)
v = [1.0, 2.0, 3.0]
@test Ma*v ≈ M.R * v

Test Passed

In [95]:
# Test multiplication of a EulerAngle instance and a RVector (SVector, QVector)
M = RotationMatrix(Rp)
Ma = EulerAngle(M)
v = RVector([1.0, 2.0, 3.0])
@test Ma*v ≈ M.R * v

Test Passed